In [1]:
import PIL
import gc
import torch
import torchvision
import os

import numpy as np
import matplotlib.pyplot as plt  
import torch.nn as nn 
import torch.optim as optim
import torch.nn.functional as F

from torchvision import datasets, transforms
from torch.utils.data import Subset
from IPython.core.display import display, HTML
from numpy.random import RandomState
from wide_resnet import WideResNet
from auto_augment import AutoAugment, Cutout
from efficientnet_pytorch import EfficientNet
from cifar_loader import SmallSampleController
import torchvision.models as models


# display(HTML("<style>.container { width:40% !important; }</style>"))


In [2]:

def getAcc(preds,targets):
    return np.sum([1 if preds[i] == targets[i] else 0 for i in range(len(preds))])/len(preds)

def train(model, device, train_loader, optimizer, epoch, display=True):
    """
    Summary: Implements the training procedure for a given model
    == params ==
    model: the model to test
    device: cuda or cpu 
    optimizer: the optimizer for our training
    train_loader: dataloader for our train data
    display: output flag
    == output ==
    the mean train loss, the train accuracy
    """
    
    lossTracker = []
    
    targets=[]
    preds=[]
    
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        lossTracker.append(loss.detach())
        with torch.no_grad():
            pred = torch.argmax(output,1).cpu().numpy()

            preds.extend(pred)
            targets.extend(target.cpu().numpy())
        
    lossTracker = [x.item() for x in lossTracker]
    meanLoss = np.mean(lossTracker)
    accuracy = getAcc(preds,targets)
    if display:
        print('Train Epoch: {} [acc: {:.0f}%]\tLoss: {:.6f}'.format(
          epoch, 100. * accuracy, meanLoss))
        
    return accuracy, meanLoss



def test(model, device, test_loader,verbose=True):
    """
    Summary: Implements the testing procedure for a given model
    == params ==
    model: the model to test
    device: cuda or cpu 
    test_loader: dataloader for our test data
    verbose: output flag
    == output ==
    the mean test loss, the test accuracy
    """
    
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, size_average=False).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            

    meanLoss = test_loss / len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    
    if verbose: print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        mean_test_loss, correct, len(test_loader.dataset),
        accuracy))
        
    return accuracy, meanLoss


def checkTest(model,device,valSets,valTracker,latexTracker,epoch,
              model_name,optim_name,lr,totalTestSamples,seed,verbose=True):
    """
    Summary: checks the test accuracy, prints, and saves statistics
    """
    tempAcc = []
    tempLoss = []
    for val_loader in valSets:
        acc,loss = test(model, device, val_loader,verbose = False)
        tempAcc.append(acc)
        tempLoss.append(loss)
        
    meanAcc = np.mean(tempAcc)
    stdAcc = np.std(tempAcc)
    
    meanLoss = np.mean(tempLoss)
    if verbose:
        print('[Trained for {} epochs and tested on {} sets of 2000 images]\
        Avg Acc: {:.2f} +- {:.2f} , Avg Loss: {:.2f}'.format(
            epoch,VALIDATION_SET_NUM,meanAcc,stdAcc,meanLoss))
        
        
    tableRow = getLatexRow(architecture=model_name,epoch=epoch,accuracy=meanAcc,optim=optim_name,
                           lr=lr,totalTestSamples=totalTestSamples,dataAug="Nothing",
                           seed=seed,title=False)
    
    latexTracker.append(tableRow)
        
    valTracker["allLoss"].extend(tempLoss)
    valTracker["allAcc"].extend(tempAcc)
    valTracker["meanLoss"].append(meanLoss)
    valTracker["meanAcc"].append(meanAcc)
    valTracker["stdAcc"].append(stdAcc)





In [3]:
def getLatexRow(architecture,epoch,accuracy,optim,lr,
                totalTestSamples,dataAug,seed,title=False):
    """
    Summary: generates one row of latex for a results table
    """
    categories = ["Model","Epoch","Accuracy","Optimizer","lr","Test Sample Num",
                  "data augmentation","seed"]
    row = [str(architecture),str(epoch),str(round(accuracy,3)),str(optim),
           str(lr),str(totalTestSamples),str(dataAug),str(seed)]
    
    if title:
        c = "&".join(categories)
        r = "&".join(row)
        return "{}\\\\\n{}\\\\".format(c,r)
    else:
        r = "&".join(row)
        return "{}\\\\".format(r)
    
    
def plot(xlist,ylist,xlab,ylab,title,color,label,savedir=".",save=False):
    """
    Summary: plots the given list of numbers against its idices and 
    allows for high resolution saving
    """
    fig = plt.figure()
    plt.title(title)
    plt.xlabel(xlab)
    plt.ylabel(ylab)
    plt.plot(xlist,ylist,color=color,marker=".",label=label)
    plt.legend()
    
    if save:
        if not os.path.isdir(savedir):
            os.mkdir(savedir)
        filepath = os.path.join(savedir,"{}".format(title))
        plt.savefig(filepath+".pdf")
        os.system("pdftoppm -png -r 300 {}.pdf {}.png".format(filepath,filepath))
        
    plt.show()
    
    

In [4]:
def getModel(model_name):
    if "wide" in model_name.lower():
        return WideResNet(28, 10, num_classes=10)
    elif "efficient" in model_name.lower():
        return EfficientNet.from_pretrained(model_name,num_classes = 10) # change to not be pretrained
    elif "vgg16" in model_name.lower():
        model = models.vgg16(pretrained=True)
#         model.classifier[6] = nn.Linear(4096, 10)
        return model
    elif "alexnet" in model_name.lower():
        model = models.alexnet(pretrained=True)
#         model.classifier = nn.Linear(256 * 6 * 6, 10)
        return model
    elif "resnet18" in model_name.lower():
        model = models.resnet18(pretrained=True)
#         model.fc.out_features = 10
        return model
    elif "resnet50" in model_name.lower():
        model = models.resnet50(pretrained=True)
#         model.fc.out_features = 10
        return model
    elif "densenet161" in model_name.lower():
        model = models.densenet161(pretrained=True)
#         model.fc.out_features = 10
        return model
    elif "wideresnet" in model_name.lower():
            model = models.wide_resnet50_2(pretrained=True)
            return model
    elif "resnext101" in model_name.lower():
            model = models.resnext101_32x8d(pretrained=True)
            return model
    elif "inception_v3" in model_name.lower():
            model = models.inception_v3(pretrained=True,aux_logits=False)
            return model
    elif "squeezenet" in model_name.lower():
        model = models.squeezenet1_0(pretrained=True)
        model.classifier[1] = nn.Conv2d(512, 10, kernel_size=(1,1), stride=(1,1))
        return model
    
    
    
def getOptimizer128(optimizer_name,parameters):
    if "sgd" in  optimizer_name.lower():
        LR = 0.1
        optim = torch.optim.SGD(parameters, 
                                  lr=LR, momentum=0.9,
                                  weight_decay=0.0005)
        return optim, LR
    elif "adam" in optimizer_name.lower():
        LR = 0.00005
        optim = torch.optim.Adam(parameters, 
                              lr=LR, weight_decay=0)
        return optim, LR
        
    

In [5]:
class featureExtractor(nn.Module):
    def __init__(self,efficientNet):
        super().__init__()
        self.eNet = efficientNet 
        self._avg_pooling = nn.AdaptiveAvgPool2d(1)
        self._dropout = nn.Dropout(self.eNet._global_params.dropout_rate)
        self._fc = nn.Linear(2560,10)
        
        
    def forward(self,inputs):
        x = self.eNet.extract_features(inputs)
        x = x.flatten(start_dim=1)
        x = self._dropout(x)
        x = self._fc(x)
        return x

In [8]:
torch.cuda.empty_cache()
gc.collect()

#     resnet18 = models.resnet18()
#     alexnet = models.alexnet()
#     vgg16 = models.vgg16()
#     squeezenet = models.squeezenet1_0()
#     densenet = models.densenet161()
#     inception = models.inception_v3()
#     googlenet = models.googlenet()
#     shufflenet = models.shufflenet_v2_x1_0()
#     mobilenet = models.mobilenet_v2()
#     resnext50_32x4d = models.resnext50_32x4d()
#     wide_resnet50_2 = models.wide_resnet50_2()
#     mnasnet = models.mnasnet1_0()

OPTIM = "Adam"
MODEL = "squeezenet"
EPOCH_NUM = 1000
TRAIN_SAMPLE_NUM = 100
VAL_SAMPLE_NUM = 2000
BATCH_SIZE = 64
VALIDATION_SET_NUM = 5
AUGMENT = True
VAL_DISPLAY_DIVISOR =25
CIFAR_TRAIN = True

#cifar-10:
#mean = (0.4914, 0.4822, 0.4465)
#std = (0.247, 0.243, 0.261)

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])
if AUGMENT:
    dataAugmentation = [ 
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        AutoAugment(),
        Cutout()
    ]
    augment = "Crop,Flip,AutoAugment,Cutout"
else: 
    dataAugmentation = []
    augment = "Nothing"


# We resize images to allow using imagenet pre-trained models, is there a better way?
# resize = transforms.Resize(299) 

transform_train = transforms.Compose(dataAugmentation + [transforms.ToTensor(), normalize]) 
transform_val = transforms.Compose([transforms.ToTensor(), normalize]) #careful to keep this one same

cifar_train = datasets.CIFAR10(root='.',train=CIFAR_TRAIN, transform=transform_train, download=True)
cifar_val = datasets.CIFAR10(root='.',train=CIFAR_TRAIN, transform=transform_val, download=True)

ss = SmallSampleController(numClasses=10,trainSampleNum=TRAIN_SAMPLE_NUM, # abstract the data-loading procedure
                           valSampleNum=VAL_SAMPLE_NUM, batchSize=BATCH_SIZE, 
                           multiplier=VALIDATION_SET_NUM, trainDataset=cifar_train, 
                           valDataset=cifar_val)
    
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_data, valSets, seed = ss.generateNewSet(device,valMultiplier = VALIDATION_SET_NUM) #Sample from datasets



eNet = getModel(MODEL).cuda()

# for param in eNet.parameters():
#     param.requires_grad = False
    
# model = featureExtractor(eNet).cuda()
model = eNet

# model.fc.out_features = 10
    
optimizer,LR = getOptimizer128(OPTIM,model.classifier.parameters())

# print(model.parameters)

print(' => Total trainable parameters: %.2fM' % (sum(p.numel() for p in model.parameters()) / 1000000.0))        

trainTracker = {"meanLoss":[],"accuracy":[]}
valTracker = {"allLoss":[],"allAcc":[],"meanLoss":[],"meanAcc":[],"stdAcc":[]}
latexTracker = []

print("Begin Train for {} epochs".format(EPOCH_NUM))
for epoch in range(EPOCH_NUM):
    acc, loss = train(model, device, train_data[0], optimizer, epoch+1, display=True)
    trainTracker["meanLoss"].append(loss)
    trainTracker["accuracy"].append(acc)
    
    if (epoch+1) % VAL_DISPLAY_DIVISOR == 0:
        checkTest(model,device,valSets,valTracker,latexTracker,epoch+1,
              model_name=MODEL,optim_name=OPTIM,lr=LR,totalTestSamples=VAL_SAMPLE_NUM*VALIDATION_SET_NUM,
                  seed=seed,verbose=True)
        
          
        
        

Files already downloaded and verified
Files already downloaded and verified
Generated new permutation of the CIFAR train dataset with                 seed:1619995059, train sample num: 100, test sample num: 2000
 => Total trainable parameters: 0.74M
Begin Train for 1000 epochs
Train Epoch: 1 [acc: 8%]	Loss: 14.944014
Train Epoch: 2 [acc: 11%]	Loss: 14.469419
Train Epoch: 3 [acc: 11%]	Loss: 14.141998
Train Epoch: 4 [acc: 9%]	Loss: 12.489553
Train Epoch: 5 [acc: 12%]	Loss: 12.887087
Train Epoch: 6 [acc: 11%]	Loss: 14.108051
Train Epoch: 7 [acc: 7%]	Loss: 14.177855
Train Epoch: 8 [acc: 7%]	Loss: 13.686115
Train Epoch: 9 [acc: 6%]	Loss: 13.467068
Train Epoch: 10 [acc: 9%]	Loss: 11.572503
Train Epoch: 11 [acc: 10%]	Loss: 12.283890
Train Epoch: 12 [acc: 7%]	Loss: 13.040585
Train Epoch: 13 [acc: 12%]	Loss: 12.436163
Train Epoch: 14 [acc: 13%]	Loss: 12.887883
Train Epoch: 15 [acc: 10%]	Loss: 12.785753
Train Epoch: 16 [acc: 6%]	Loss: 13.504804
Train Epoch: 17 [acc: 9%]	Loss: 11.611463
Train Epo

Train Epoch: 173 [acc: 13%]	Loss: 4.605859
Train Epoch: 174 [acc: 10%]	Loss: 4.808379
Train Epoch: 175 [acc: 16%]	Loss: 4.155921
[Trained for 175 epochs and tested on 5 sets of 2000 images]        Avg Acc: 12.99 +- 0.48 , Avg Loss: 3.00
Train Epoch: 176 [acc: 10%]	Loss: 4.592595
Train Epoch: 177 [acc: 11%]	Loss: 4.650919
Train Epoch: 178 [acc: 12%]	Loss: 4.732216
Train Epoch: 179 [acc: 11%]	Loss: 4.696495
Train Epoch: 180 [acc: 8%]	Loss: 3.780921
Train Epoch: 181 [acc: 18%]	Loss: 4.378783
Train Epoch: 182 [acc: 13%]	Loss: 4.215981
Train Epoch: 183 [acc: 10%]	Loss: 4.323562
Train Epoch: 184 [acc: 11%]	Loss: 3.693515
Train Epoch: 185 [acc: 12%]	Loss: 4.305671
Train Epoch: 186 [acc: 12%]	Loss: 4.249485
Train Epoch: 187 [acc: 9%]	Loss: 3.356219
Train Epoch: 188 [acc: 6%]	Loss: 4.368622
Train Epoch: 189 [acc: 16%]	Loss: 3.732527
Train Epoch: 190 [acc: 8%]	Loss: 4.014707
Train Epoch: 191 [acc: 10%]	Loss: 3.645141
Train Epoch: 192 [acc: 12%]	Loss: 4.333823
Train Epoch: 193 [acc: 12%]	Loss: 4.

Train Epoch: 348 [acc: 11%]	Loss: 2.688836
Train Epoch: 349 [acc: 6%]	Loss: 2.898606
Train Epoch: 350 [acc: 13%]	Loss: 2.654753
[Trained for 350 epochs and tested on 5 sets of 2000 images]        Avg Acc: 12.05 +- 0.19 , Avg Loss: 2.39
Train Epoch: 351 [acc: 10%]	Loss: 2.504740
Train Epoch: 352 [acc: 13%]	Loss: 2.385286
Train Epoch: 353 [acc: 12%]	Loss: 2.601981
Train Epoch: 354 [acc: 12%]	Loss: 2.491493
Train Epoch: 355 [acc: 12%]	Loss: 2.883533
Train Epoch: 356 [acc: 13%]	Loss: 2.949444
Train Epoch: 357 [acc: 9%]	Loss: 2.529346
Train Epoch: 358 [acc: 9%]	Loss: 2.713572
Train Epoch: 359 [acc: 13%]	Loss: 2.455952
Train Epoch: 360 [acc: 13%]	Loss: 2.818832
Train Epoch: 361 [acc: 9%]	Loss: 2.628139
Train Epoch: 362 [acc: 11%]	Loss: 3.033123
Train Epoch: 363 [acc: 12%]	Loss: 2.836163
Train Epoch: 364 [acc: 10%]	Loss: 2.581667
Train Epoch: 365 [acc: 11%]	Loss: 2.580299
Train Epoch: 366 [acc: 10%]	Loss: 2.679166
Train Epoch: 367 [acc: 10%]	Loss: 2.948472
Train Epoch: 368 [acc: 9%]	Loss: 2.7

Train Epoch: 522 [acc: 9%]	Loss: 2.563887
Train Epoch: 523 [acc: 10%]	Loss: 2.700627
Train Epoch: 524 [acc: 8%]	Loss: 2.425941
Train Epoch: 525 [acc: 9%]	Loss: 2.327449
[Trained for 525 epochs and tested on 5 sets of 2000 images]        Avg Acc: 11.25 +- 0.17 , Avg Loss: 2.32
Train Epoch: 526 [acc: 11%]	Loss: 2.382989
Train Epoch: 527 [acc: 13%]	Loss: 2.509565
Train Epoch: 528 [acc: 10%]	Loss: 2.482944
Train Epoch: 529 [acc: 9%]	Loss: 2.498296
Train Epoch: 530 [acc: 11%]	Loss: 2.461454
Train Epoch: 531 [acc: 11%]	Loss: 2.418592
Train Epoch: 532 [acc: 9%]	Loss: 2.353960
Train Epoch: 533 [acc: 11%]	Loss: 2.347179
Train Epoch: 534 [acc: 10%]	Loss: 2.334309
Train Epoch: 535 [acc: 10%]	Loss: 2.438868
Train Epoch: 536 [acc: 14%]	Loss: 2.535753
Train Epoch: 537 [acc: 11%]	Loss: 2.383984
Train Epoch: 538 [acc: 10%]	Loss: 2.545972
Train Epoch: 539 [acc: 10%]	Loss: 2.661451
Train Epoch: 540 [acc: 9%]	Loss: 2.586443
Train Epoch: 541 [acc: 13%]	Loss: 2.419617
Train Epoch: 542 [acc: 13%]	Loss: 2.28

Train Epoch: 696 [acc: 8%]	Loss: 2.487889
Train Epoch: 697 [acc: 12%]	Loss: 2.397910
Train Epoch: 698 [acc: 9%]	Loss: 2.425637
Train Epoch: 699 [acc: 12%]	Loss: 2.390606
Train Epoch: 700 [acc: 10%]	Loss: 2.529527
[Trained for 700 epochs and tested on 5 sets of 2000 images]        Avg Acc: 11.11 +- 0.15 , Avg Loss: 2.31
Train Epoch: 701 [acc: 11%]	Loss: 2.386973
Train Epoch: 702 [acc: 10%]	Loss: 2.490532
Train Epoch: 703 [acc: 10%]	Loss: 2.366021
Train Epoch: 704 [acc: 11%]	Loss: 2.404841
Train Epoch: 705 [acc: 11%]	Loss: 2.407950
Train Epoch: 706 [acc: 13%]	Loss: 2.388613
Train Epoch: 707 [acc: 10%]	Loss: 2.435801
Train Epoch: 708 [acc: 11%]	Loss: 2.256011
Train Epoch: 709 [acc: 10%]	Loss: 2.370331
Train Epoch: 710 [acc: 11%]	Loss: 2.380036
Train Epoch: 711 [acc: 11%]	Loss: 2.340738
Train Epoch: 712 [acc: 11%]	Loss: 2.331635
Train Epoch: 713 [acc: 10%]	Loss: 2.548195
Train Epoch: 714 [acc: 10%]	Loss: 2.476269
Train Epoch: 715 [acc: 9%]	Loss: 2.400183
Train Epoch: 716 [acc: 12%]	Loss: 2

Train Epoch: 870 [acc: 11%]	Loss: 2.348017
Train Epoch: 871 [acc: 12%]	Loss: 2.286249
Train Epoch: 872 [acc: 9%]	Loss: 2.504167
Train Epoch: 873 [acc: 13%]	Loss: 2.307657
Train Epoch: 874 [acc: 10%]	Loss: 2.431086
Train Epoch: 875 [acc: 11%]	Loss: 2.345846
[Trained for 875 epochs and tested on 5 sets of 2000 images]        Avg Acc: 11.07 +- 0.19 , Avg Loss: 2.30
Train Epoch: 876 [acc: 13%]	Loss: 2.400087
Train Epoch: 877 [acc: 10%]	Loss: 2.344926
Train Epoch: 878 [acc: 10%]	Loss: 2.386945
Train Epoch: 879 [acc: 11%]	Loss: 2.381140
Train Epoch: 880 [acc: 10%]	Loss: 2.311771
Train Epoch: 881 [acc: 10%]	Loss: 2.385614
Train Epoch: 882 [acc: 9%]	Loss: 2.325305
Train Epoch: 883 [acc: 10%]	Loss: 2.304736
Train Epoch: 884 [acc: 10%]	Loss: 2.373249
Train Epoch: 885 [acc: 10%]	Loss: 2.310162
Train Epoch: 886 [acc: 10%]	Loss: 2.393570
Train Epoch: 887 [acc: 11%]	Loss: 2.345614
Train Epoch: 888 [acc: 9%]	Loss: 2.392345
Train Epoch: 889 [acc: 10%]	Loss: 2.334359
Train Epoch: 890 [acc: 13%]	Loss: 2

In [ ]:
dirname = latexTracker[-1][:-2] 

def writeTex(latexTracker,dirname):
    if not os.path.isdir(dirname):
        os.mkdir(dirname)
        
    f= open(os.path.join(dirname,"latexTable.txt"),"w")
    for x in latexTracker:
        f.write(x)
    f.close()

writeTex(latexTracker,dirname)

for x in latexTracker:
    print(x)

In [ ]:

epochList = [x+1 for x in range(len(trainTracker["meanLoss"]))]

plot(xlist=epochList,ylist=trainTracker["meanLoss"],xlab="Mean Train Loss",
    ylab="Epochs",title="Mean Train Loss over Epochs",
    color="#243A92",label="mean train loss",savedir=dirname,save=True)

plot(xlist=epochList,ylist=trainTracker["accuracy"],xlab="Train Accuracy",
    ylab="Epochs",title="Train Accuracy Over Epochs",
    color="#34267E",label="Train Accuracy",savedir=dirname,save=True)

In [ ]:

epochList = [VAL_DISPLAY_DIVISOR*(x+1) for x in range(len(valTracker["meanLoss"]))]

plot(xlist=epochList,ylist=valTracker["meanLoss"],xlab="Epochs",
    ylab="Mean Val Loss",title="Mean Val Loss over Epochs",
    color="#243A92",label="mean val loss",savedir=dirname,save=True)

plot(xlist=epochList,ylist=valTracker["meanAcc"],xlab="Epochs",
    ylab="Val Accuracy",title="Val Accuracy Over Epochs",
    color="#34267E",label="Val Accuracy",savedir=dirname,save=True)

plot(xlist=epochList,ylist=valTracker["stdAcc"],xlab="Epochs",
    ylab="Val Accuracy Standard Deviation",title="Val Accuracy Standard Deviation Over Epochs",
    color="#34267E",label="Val Accuracy SD",savedir=dirname,save=True)


valSetEvalCount = VAL_DISPLAY_DIVISOR * EPOCH_NUM * VALIDATION_SET_NUM
epochList = [VAL_DISPLAY_DIVISOR*(x+1) for x in range(len(valTracker["meanLoss"]))\
             for y in range(VALIDATION_SET_NUM)]


plot(xlist=epochList,ylist=valTracker["allLoss"],xlab="Val Set Evaluations",
    ylab="Val Loss",title="Val loss over val set evaluations ({} \
every {} epochs)".format(VALIDATION_SET_NUM,VAL_DISPLAY_DIVISOR),
    color="#34267E",label="Val Loss",savedir=dirname,save=True)

plot(xlist=epochList,ylist=valTracker["allAcc"],xlab="Val Set Evaluations",
    ylab="Val Accuracy",title="Val loss over val set evaluations ({} \
every {} epochs) ".format(VALIDATION_SET_NUM,VAL_DISPLAY_DIVISOR),
    color="#34267E",label="Val Accuracy",savedir=dirname,save=True)